#  1. ADIM – Gerekli Kütüphaneleri İçe Aktar

In [97]:
import subprocess
import tkinter as tk
from tkinter import scrolledtext
from datetime import datetime
import csv


# 2. ADIM – PowerShell ile Güncellemeleri Al

In [100]:
def guncellemeleri_al():
    try:
        output = subprocess.check_output(
            ["powershell", "-Command", "Get-HotFix | Sort-Object InstalledOn -Descending"],
            stderr=subprocess.STDOUT
        ).decode("utf-8", errors="ignore")

        # Satırları işle
        lines = [line.strip() for line in output.splitlines() if line.strip()]
        updates = []
        current = {}
        for line in lines:
            if "Source" in line and current:
                updates.append(current)
                current = {}
            parts = line.split(":", 1)
            if len(parts) == 2:
                key, value = parts
                current[key.strip()] = value.strip()
        if current:
            updates.append(current)

        return updates
    except subprocess.CalledProcessError:
        return []


# 3. ADIM – Son 5 Güncellemeyi Al

In [103]:
def son_5_guncellemeyi_getir(updates):
    return updates[:5]


# 4. ADIM – Sistem Güncel mi Kontrol Et

In [106]:
def guncel_mi(updates):
    try:
        last_date = updates[0].get("InstalledOn", "").split(" ")[0]  # 🔧 saat kısmını at
        date_obj = datetime.strptime(last_date, "%d.%m.%Y")
        fark = (datetime.now() - date_obj).days
        return fark <= 15, date_obj.strftime("%Y-%m-%d"), fark
    except:
        return False, "Tarih alınamadı", "?"


# 5. ADIM – CSV Loglama Fonksiyonu

In [109]:
def log_yaz(updates):
    with open("patch_log.csv", "a", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["LogTime", "HotFixID", "InstalledOn", "Description"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for upd in updates:
            writer.writerow({
                "LogTime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "HotFixID": upd.get("HotFixID", ""),
                "InstalledOn": upd.get("InstalledOn", ""),
                "Description": upd.get("Description", "")
            })


# 6. ADIM – Security Güncellemeleri için Filtreleme

In [112]:
def filtrele_security(updates):
    return [u for u in updates if "security" in u.get("Description", "").lower()]


# 7. ADIM – GUI Arayüz (Tkinter)

In [115]:
pencere = tk.Tk()
pencere.title("📦 Windows Patch Kontrol Aracı")
pencere.geometry("750x500")

etiket = tk.Label(pencere, text="Yüklü Güncellemeler (Son 5)", font=("Arial", 14))
etiket.pack(pady=10)

renk_kutu = tk.Label(pencere, width=30, height=2, bg="gray", relief="ridge", bd=4)
renk_kutu.pack(pady=5)

kutucuk = scrolledtext.ScrolledText(pencere, width=90, height=15, font=("Consolas", 9))
kutucuk.pack(padx=10, pady=5)

durum_etiketi = tk.Label(pencere, text="", font=("Arial", 12, "bold"))
durum_etiketi.pack(pady=5)

security_var = tk.IntVar()
checkbtn = tk.Checkbutton(pencere, text="Sadece Güvenlik Güncellemelerini Göster", variable=security_var)
checkbtn.pack()


# 8. ADIM – Buton Fonksiyonu (Göster)

In [118]:
def goster():
    updates = guncellemeleri_al()
    if not updates:
        kutucuk.insert(tk.END, "Güncelleme bilgileri alınamadı.")
        return

    log_yaz(updates)
    son5 = son_5_guncellemeyi_getir(updates)
    if security_var.get():
        son5 = filtrele_security(son5)

    kutucuk.delete("1.0", tk.END)
    for u in son5:
        for k, v in u.items():
            kutucuk.insert(tk.END, f"{k}: {v}\n")
        kutucuk.insert(tk.END, "-"*60 + "\n")

    guncel, tarih, fark = guncel_mi(updates)
    if guncel:
        durum_etiketi.config(text=f"✅ Sistem güncel. Son güncelleme: {tarih}", fg="green")
        renk_kutu.config(bg="green")
    else:
        durum_etiketi.config(text=f"⚠️ Sistem güncel değil! Son güncelleme: {tarih} ({fark} gün önce)", fg="red")
        renk_kutu.config(bg="red")


# 9. ADIM – Göster Butonu ve GUI Döngüsü

In [ ]:
buton = tk.Button(pencere, text="Güncellemeleri Göster", command=goster, bg="lightblue")
buton.pack(pady=10)

pencere.mainloop()
